In [1]:
import pandas as pd
import numpy as np
import bqplot
import traitlets
import ipywidgets

In [2]:
buildings = pd.read_csv("building_inventory.csv", 
                        na_values={"Year Acquired": 0,
                                   "Year Constructed": 0,
                                   "Square Footage": 0})

buildings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8862 entries, 0 to 8861
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Agency Name              8862 non-null   object 
 1   Location Name            8862 non-null   object 
 2   Address                  8811 non-null   object 
 3   City                     8862 non-null   object 
 4   Zip code                 8862 non-null   int64  
 5   County                   8837 non-null   object 
 6   Congress Dist            8862 non-null   int64  
 7   Congressional Full Name  8699 non-null   object 
 8   Rep Dist                 8862 non-null   int64  
 9   Rep Full Name            8839 non-null   object 
 10  Senate Dist              8862 non-null   int64  
 11  Senator Full Name        8839 non-null   object 
 12  Bldg Status              8862 non-null   object 
 13  Year Acquired            8597 non-null   float64
 14  Year Constructed        

In [3]:
# Show values in Year Acquired column
buildings.head()[['Year Acquired']]

,Year Acquired
0,1975.0
1,2004.0
2,2004.0
3,2004.0
4,2004.0


In [4]:
# Convert without format
buildings_cp = buildings.copy()
buildings_cp['Year Acquired'] = pd.to_datetime(buildings_cp['Year Acquired'])
buildings_cp.head()[['Year Acquired']]

,Year Acquired
0,1970-01-01 00:00:00.000001975
1,1970-01-01 00:00:00.000002004
2,1970-01-01 00:00:00.000002004
3,1970-01-01 00:00:00.000002004
4,1970-01-01 00:00:00.000002004


In [5]:
buildings_cp['Year Acquired'].dt.year.tolist()[:3]

[1970.0, 1970.0, 1970.0]

In [6]:
buildings_cp['Year Acquired'].dt.microsecond.tolist()[:3]

[1.0, 2.0, 2.0]

In [7]:
# Convert with format
buildings['Year Acquired'] = pd.to_datetime(buildings['Year Acquired'], 
                                            format='%Y')
print(buildings['Year Acquired'].dtype)
buildings.head()[['Year Acquired']]

datetime64[ns]


,Year Acquired
0,1975-01-01
1,2004-01-01
2,2004-01-01
3,2004-01-01
4,2004-01-01


In [8]:
# Extract the year from the date object, Note the datatype
buildings['Year Acquired'] = buildings['Year Acquired'].dt.year
print(buildings['Year Acquired'].dtype)
buildings.head()[['Year Acquired']]

float64


,Year Acquired
0,1975.0
1,2004.0
2,2004.0
3,2004.0
4,2004.0


# UFO dataset

In [9]:
#!wget https://github.com/planetsig/ufo-reports/raw/master/csv-data/ufo-scrubbed-geocoded-time-standardized.csv

In [10]:
ufo = pd.read_csv('ufo-scrubbed-geocoded-time-standardized.csv', 
                  names=['date_sighted', 'city', 'state', 'country',
                         'shape', 'duration', 
                         'duration_txt', 'note', 'date_reported', 
                         'latitude', 'longitude'],
                  parse_dates=['date_sighted', 'date_reported'])
ufo = ufo.reset_index().rename(columns={'index':'ufo_id'})
print(ufo.shape)

ufo = ufo.loc[~ufo.index.isin([27822, 35692, 58591, 43782])] # Ignore dirty data for now

df = ufo.sample(n=1000, random_state=5).reset_index(drop=True)
df['date_sighted'] = df['date_sighted'].str.replace('24:00', '00:00') 
df['duration'] = df['duration'].astype(float)
df['latitude'] = df['latitude'].astype(float)
df['date_sighted'] = pd.to_datetime(df['date_sighted'])

print(df.info())
df.head()

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(80332, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ufo_id         1000 non-null   int64         
 1   date_sighted   1000 non-null   datetime64[ns]
 2   city           1000 non-null   object        
 3   state          922 non-null    object        
 4   country        866 non-null    object        
 5   shape          974 non-null    object        
 6   duration       1000 non-null   float64       
 7   duration_txt   1000 non-null   object        
 8   note           1000 non-null   object        
 9   date_reported  1000 non-null   datetime64[ns]
 10  latitude       1000 non-null   float64       
 11  longitude      1000 non-null   float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(6)
memory usage: 93.9+ KB
None


,ufo_id,date_sighted,city,state,country,shape,duration,duration_txt,note,date_reported,latitude,longitude
0,60887,2010-07-30 23:05:00,uk/scotland,NaN,NaN,light,180.0,3 mins,two bright lights travelling at speed across a...,2010-08-24,56.490671,-4.202646
1,5167,2011-10-28 16:39:00,scottsdale,az,us,unknown,300.0,5min or less,Three to Five comet-like objects seen descendi...,2011-12-12,33.509167,-111.898333
2,45720,2007-05-05 01:00:00,denton,ne,us,other,420.0,5-7 minutes,Object seen outside of Lincoln &#44NE,2007-06-12,40.737778,-96.844167
3,40365,2006-04-05 22:25:00,oklahoma city,ok,us,circle,600.0,10 minutes,Two hovering orange circles drop third orange ...,2006-05-15,35.467500,-97.516111
4,28803,2003-02-27 00:00:00,patoka,in,us,unknown,300.0,5 minutes off and on,on a morning jog me and my friend saw lights i...,2003-03-21,38.406944,-87.585556


# Time-series data

In [11]:
# prep data: Number of UFOs sighted in each month
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

line_data = df.groupby(pd.Grouper(key='date_sighted', 
                                  freq='M'))[['ufo_id']].count()
line_data

,ufo_id
date_sighted,
1955-06-30,1
1955-07-31,0
1955-08-31,0
1955-09-30,0
1955-10-31,0
...,...
2014-01-31,8
2014-02-28,2
2014-03-31,8


In [12]:
# Check if grouped result is correct
df.loc[df['date_sighted'].dt.year == 1955]

,ufo_id,date_sighted,city,state,country,shape,duration,duration_txt,note,date_reported,latitude,longitude
330,48587,1955-06-15 04:30:00,missouri (northwest part),mo,NaN,light,300.0,5 min,I will never forget this this experience. The ...,2003-03-21,37.964253,-91.831833


In [13]:
line_data.loc[line_data.index.year == 1955]

,ufo_id
date_sighted,
1955-06-30,1
1955-07-31,0
1955-08-31,0
1955-09-30,0
1955-10-31,0
1955-11-30,0
1955-12-31,0


## Parameter: X lim

In [14]:
# range slider for years
my_slider = ipywidgets.SelectionRangeSlider(options=line_data.index.year,
                                            description='Year Range', 
                                            layout={'width':"600px"})
my_slider

SelectionRangeSlider(description='Year Range', index=(0, 0), layout=Layout(width='600px'), options=(1955, 1955…

In [15]:
my_slider.value

()

In [16]:
# A line plot + displayed the selected year window

# Scale
x_sc = bqplot.DateScale()
y_sc = bqplot.LinearScale()

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Date Sighted')
y_ax = bqplot.Axis(scale=y_sc, label='UFO counts', 
                   orientation='vertical')

# Mark
lines = bqplot.Lines(x=line_data.index, 
                     y=line_data['ufo_id'], 
                     scales={'x':x_sc, 'y':y_sc})

# Interaction
my_slider = ipywidgets.SelectionRangeSlider(options=line_data.index.year,
                                            description='Year Range', 
                                            layout={'width':"600px"})

def slider_func(change):
    yr_range = my_slider.value
    #print(yr_range)
    
    yr_start, yr_end = yr_range
    
    filter_1 = line_data.index.year >= yr_start
    filter_2 = line_data.index.year <= yr_end
    
    line_data_selected = line_data.loc[(filter_1)&(filter_2)]
    
    lines.x = line_data_selected.index
    lines.y = line_data_selected['ufo_id']

my_slider.observe(slider_func, 'value')

# Fig
line_fig = bqplot.Figure(marks=[lines], axes=[x_ax, y_ax])
line_fig.layout.height='200px'
line_fig_slider = ipywidgets.VBox([my_slider, line_fig])
line_fig_slider

# Heatmap and click selected

In [17]:
# Prep data: UFOs reported in different years and countries
heatmap_data = df.groupby([df['date_reported'].dt.year, 'country'])[['ufo_id']].count()
heatmap_data = heatmap_data.reset_index()
heatmap_data = heatmap_data.pivot(index='date_reported', 
                                  columns='country', 
                                  values='ufo_id')
heatmap_data

country,au,ca,de,gb,us
date_reported,,,,,
1998,NaN,NaN,NaN,NaN,9.0
1999,NaN,NaN,NaN,NaN,50.0
2000,1.0,3.0,NaN,NaN,25.0
2001,NaN,2.0,NaN,2.0,31.0
2002,NaN,5.0,NaN,5.0,39.0
2003,1.0,2.0,NaN,3.0,47.0
2004,3.0,3.0,NaN,NaN,47.0
2005,NaN,NaN,NaN,4.0,55.0
2006,NaN,2.0,1.0,NaN,30.0


In [18]:
# Heatmap without interactivity
# Scale
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

clr = np.log10(heatmap_data)
c_sc = bqplot.ColorScale(scheme='BuPu', min=np.nanmin(clr) , max=np.nanmax(clr))

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Country')
y_ax = bqplot.Axis(scale=y_sc, label='Year Reported', orientation='vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, side='right')

# Marks
heatmap = bqplot.GridHeatMap(column=heatmap_data.columns,
                             row=heatmap_data.index,
                             scales={'column':x_sc, 'row':y_sc, 'color':c_sc}, 
                             color=clr)

# Fig
heatmap_fig = bqplot.Figure(marks=[heatmap], axes=[x_ax, y_ax, c_ax])
heatmap_fig

Figure(axes=[Axis(label='Country', scale=OrdinalScale()), Axis(label='Year Reported', orientation='vertical', …

In [19]:
# Heatmap with click and select that shows the UFO counts in each month

# Left plot: heatmap
# Scale
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

clr = np.log10(heatmap_data)
c_sc = bqplot.ColorScale(scheme='BuPu', min=np.nanmin(clr) , max=np.nanmax(clr))

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Country')
y_ax = bqplot.Axis(scale=y_sc, label='Year Reported', orientation='vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, side='right')

# Marks
heatmap = bqplot.GridHeatMap(column=heatmap_data.columns,
                             row=heatmap_data.index,
                             scales={'column':x_sc, 'row':y_sc, 'color':c_sc}, 
                             color=clr, 
                             interactions={'click':'select'}, 
                             selected_style={'fill':'green'})


# Right plot: line plot

# Scale 
x_sc_r = bqplot.DateScale()
y_sc_r = bqplot.LinearScale()

# Axis
x_ax_r = bqplot.Axis(scale=x_sc_r, label='Month')
y_ax_r = bqplot.Axis(scale=y_sc_r, label='UFO counts', 
                     orientation='vertical')

# Mark
lines = bqplot.Lines(scales={'x':x_sc_r, 'y':y_sc_r})


# Interaction:

def observe_func(change):
    selected_cell = heatmap.selected
    print(selected_cell)
    
    selected_row, selected_col = selected_cell[0]
    
    years = heatmap_data.index.tolist()
    countries = heatmap_data.columns.tolist()
    
    selected_yr = years[selected_row]
    selected_country = countries[selected_col]
    
    print(selected_yr, selected_country)
    
    filter_1 = df['date_reported'].dt.year == selected_yr
    filter_2 = df['country'] == selected_country
    
    df_selected = df.loc[(filter_1)&(filter_2)]
    
    df_selected_g = df_selected.groupby(pd.Grouper(key='date_reported', 
                                                   freq='M'))[['ufo_id']].count()
    
    lines.x = df_selected_g.index
    lines.y = df_selected_g['ufo_id']

heatmap.observe(observe_func, 'selected')


# Fig
heatmap_fig = bqplot.Figure(marks=[heatmap], axes=[x_ax, y_ax, c_ax])
line_fig = bqplot.Figure(marks=[lines], axes=[x_ax_r, y_ax_r], 
                         title='UFO counts by month in the selected year and country')

heatmap_fig.layout.width = '500px'
line_fig.layout.width = '500px'

# Dashboard
my_dashboard = ipywidgets.HBox([heatmap_fig, line_fig])
my_dashboard

# Maps

In [20]:
# US map data
us_map_data = bqplot.topo_load('map_data/USStatesMap.json')
us_map_data.keys()

dict_keys(['type', 'objects', 'transform', 'arcs'])

In [21]:
# Data of one state
us_map_data['objects']['subunits']['geometries'][0]

{'arcs': [[[83,
    78,
    81,
    -22,
    79,
    -113,
    234,
    235,
    -352,
    381,
    382,
    -518,
    562,
    493,
    534,
    535,
    537,
    482,
    483,
    669,
    670,
    671,
    672,
    673,
    580,
    581,
    668,
    -637,
    585,
    586,
    588,
    454,
    330,
    221,
    175,
    222,
    306,
    264,
    358,
    261,
    359,
    308,
    394,
    357,
    294,
    184,
    172,
    185,
    126,
    2,
    73]],
  [[116]],
  [[117]],
  [[118]],
  [[173]],
  [[259]],
  [[291]]],
 'type': 'MultiPolygon',
 'id': 53,
 'properties': {'name': 'Washington'}}

In [22]:
# US Map with tooltip

# Data
us_map_data = bqplot.topo_load('map_data/USStatesMap.json')

# Scale
geo_sc = bqplot.AlbersUSA()

# Axis

# Mark
us_map = bqplot.Map(map_data = us_map_data, 
                    scales={'projection':geo_sc})


# Interaction: tooltip
us_map.tooltip = bqplot.Tooltip(fields=['id', 'name'])

# Fig
us_map_fig = bqplot.Figure(marks=[us_map], 
                           fig_margin={'top':0, 'bottom':0, 'left':0, 'right':0})
us_map_fig

Figure(fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0}, marks=[Map(hovered_styles={'hovered_fill': '…

In [23]:
# World Map with tooltip

# Data
world_map_data = bqplot.topo_load('map_data/WorldMap.json')

# Scale
world_geo_sc = bqplot.Mercator()

# Axis

# Mark
world_map = bqplot.Map(map_data=world_map_data, 
                       scales={'projection': world_geo_sc})

# Interaction
world_map.tooltip = bqplot.Tooltip(fields=['id', 'name'])

# Fig
world_map_fig = bqplot.Figure(marks=[world_map],
                              fig_margin={'top':0, 'left':0, 'bottom':0, 'right':0})
world_map_fig

Figure(fig_margin={'top': 0, 'left': 0, 'bottom': 0, 'right': 0}, marks=[Map(hovered_styles={'hovered_fill': '…

# US Map with customized colors

In [24]:
df.head()

,ufo_id,date_sighted,city,state,country,shape,duration,duration_txt,note,date_reported,latitude,longitude
0,60887,2010-07-30 23:05:00,uk/scotland,NaN,NaN,light,180.0,3 mins,two bright lights travelling at speed across a...,2010-08-24,56.490671,-4.202646
1,5167,2011-10-28 16:39:00,scottsdale,az,us,unknown,300.0,5min or less,Three to Five comet-like objects seen descendi...,2011-12-12,33.509167,-111.898333
2,45720,2007-05-05 01:00:00,denton,ne,us,other,420.0,5-7 minutes,Object seen outside of Lincoln &#44NE,2007-06-12,40.737778,-96.844167
3,40365,2006-04-05 22:25:00,oklahoma city,ok,us,circle,600.0,10 minutes,Two hovering orange circles drop third orange ...,2006-05-15,35.467500,-97.516111
4,28803,2003-02-27 00:00:00,patoka,in,us,unknown,300.0,5 minutes off and on,on a morning jog me and my friend saw lights i...,2003-03-21,38.406944,-87.585556


In [25]:
!wget https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv

--2022-10-06 17:41:03--  https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 919 [text/plain]
Saving to: ‘us-state-ansi-fips.csv.6’

us-state-ansi-fips. 100%[===================>]     919  --.-KB/s    in 0s      

2022-10-06 17:41:03 (28.2 MB/s) - ‘us-state-ansi-fips.csv.6’ saved [919/919]



In [26]:
# Values in the us-state-ansi-fips.csv
# rename column names and clean values

states_df = pd.read_csv('us-state-ansi-fips.csv')
states_df.columns = ['name_full', 'fips', 'name_abbr']
states_df['name_full'] = states_df['name_full'].str.strip()
states_df['name_abbr'] = states_df['name_abbr'].str.strip().str.lower()
states_df

,name_full,fips,name_abbr
0,Alabama,1,al
1,Alaska,2,ak
2,Arizona,4,az
3,Arkansas,5,ar
4,California,6,ca
5,Colorado,8,co
6,Connecticut,9,ct
7,Delaware,10,de
8,District of Columbia,11,dc
9,Florida,12,fl


In [27]:
# FIPS and state names in the US State map data
for entry in us_map_data['objects']['subunits']['geometries']:
    fips = entry['id']
    print(fips, entry['properties'])
    

53 {'name': 'Washington'}
30 {'name': 'Montana'}
16 {'name': 'Idaho'}
38 {'name': 'North Dakota'}
27 {'name': 'Minnesota'}
23 {'name': 'Maine'}
26 {'name': 'Michigan'}
55 {'name': 'Wisconsin'}
41 {'name': 'Oregon'}
46 {'name': 'South Dakota'}
33 {'name': 'New Hampshire'}
50 {'name': 'Vermont'}
36 {'name': 'New York'}
56 {'name': 'Wyoming'}
19 {'name': 'Iowa'}
31 {'name': 'Nebraska'}
25 {'name': 'Massachusetts'}
17 {'name': 'Illinois'}
42 {'name': 'Pennsylvania'}
9 {'name': 'Connecticut'}
44 {'name': 'Rhode Island'}
6 {'name': 'California'}
49 {'name': 'Utah'}
32 {'name': 'Nevada'}
39 {'name': 'Ohio'}
18 {'name': 'Indiana'}
34 {'name': 'New Jersey'}
8 {'name': 'Colorado'}
54 {'name': 'West Virginia'}
29 {'name': 'Missouri'}
20 {'name': 'Kansas'}
10 {'name': 'Delaware'}
24 {'name': 'Maryland'}
51 {'name': 'Virginia'}
21 {'name': 'Kentucky'}
11 None
4 {'name': 'Arizona'}
40 {'name': 'Oklahoma'}
35 {'name': 'New Mexico'}
47 {'name': 'Tennessee'}
37 {'name': 'North Carolina'}
48 {'name': 'T

## Coropleth map

In [28]:
states_df.head()

,name_full,fips,name_abbr
0,Alabama,1,al
1,Alaska,2,ak
2,Arizona,4,az
3,Arkansas,5,ar
4,California,6,ca


In [29]:
states_df.loc[states_df['fips']==1]['name_abbr'].tolist()[0]

'al'

In [30]:
# Color map by number of UFOs sighted in states
clr_prep = df.loc[df['country']=='us'].groupby(['state'])[['ufo_id']].count()
#clr_prep

clr = {}
for entry in us_map_data['objects']['subunits']['geometries']:
    fips = entry['id'] 
    if fips not in [72, 78]:
        state_abbr = states_df.loc[states_df['fips']==fips]['name_abbr'].tolist()[0]
        state_clr = clr_prep.loc[clr_prep.index==state_abbr]['ufo_id'].tolist()[0]
        
        clr[fips] = state_clr

clr

{53: 44,
 30: 1,
 16: 6,
 38: 1,
 27: 17,
 23: 3,
 26: 20,
 55: 11,
 41: 18,
 46: 1,
 33: 2,
 50: 3,
 36: 42,
 56: 2,
 19: 9,
 31: 5,
 25: 13,
 17: 35,
 42: 42,
 9: 13,
 44: 2,
 6: 113,
 49: 7,
 32: 7,
 39: 25,
 18: 13,
 34: 13,
 8: 19,
 54: 7,
 29: 15,
 20: 12,
 10: 2,
 24: 13,
 51: 14,
 21: 10,
 11: 1,
 4: 42,
 40: 7,
 35: 10,
 47: 15,
 37: 24,
 48: 42,
 5: 7,
 45: 11,
 1: 8,
 13: 15,
 28: 2,
 22: 6,
 12: 41,
 15: 7,
 2: 8}

In [31]:
# Color: number of UFOs sighted in states
# Data
us_map_data = bqplot.topo_load('map_data/USStatesMap.json')

# Scale
geo_sc = bqplot.AlbersUSA()
clr_sc = bqplot.ColorScale(scheme='Oranges')

# Axis
clr_ax = bqplot.ColorAxis(scale=clr_sc, side='top')

# Mark
us_map = bqplot.Map(map_data = us_map_data, 
                    scales={'projection':geo_sc, 'color':clr_sc}, 
                    color = clr)

# Fig
us_map_fig = bqplot.Figure(marks=[us_map], axes=[clr_ax], 
                           fig_margin={'top':0, 'bottom':0, 'left':0, 'right':0})
us_map_fig

Figure(axes=[ColorAxis(scale=ColorScale(scheme='Oranges'), side='top')], fig_margin={'top': 0, 'bottom': 0, 'l…

## A Dashboard showing duration in seconds by years in the selected state

In [32]:
# Click select + sum of duration in seconds (all the selected states are combined)

# Left plot
# Scale
geo_sc = bqplot.AlbersUSA()
clr_sc = bqplot.ColorScale(scheme='Oranges')

# Axis
clr_ax = bqplot.ColorAxis(scale=clr_sc, side='top')

# Mark
us_map = bqplot.Map(map_data = us_map_data, 
                    scales={'projection':geo_sc, 'color':clr_sc}, 
                    color = clr, 
                    interactions ={'click': 'select'}, 
                    selected_styles={'selected_fill':'green'})

# Right: Line plot
# Scale
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Years')
y_ax = bqplot.Axis(scale=y_sc, label='Sum of duration in seconds', orientation='vertical')

# Marks
lines = bqplot.Lines(scales={'x':x_sc, 'y':y_sc})


# Interaction
def select_func(change):
    selected_fips = us_map.selected
    print(selected_fips)
    
    if selected_fips is not None:
    
        state_abbrs = states_df.loc[states_df['fips'].isin(selected_fips)]['name_abbr'].tolist()
        print(state_abbrs)

        df_selected = df.loc[(df['country']=='us')&(df['state'].isin(state_abbrs))]
        df_selected_g = df_selected.groupby(df_selected['date_sighted'].dt.year)[['duration']].sum()

        lines.x = df_selected_g.index
        lines.y = df_selected_g['duration']
    
    else:
        lines.x = []
        lines.y = []

us_map.observe(select_func, 'selected')

# Fig
us_map_fig = bqplot.Figure(marks=[us_map], axes=[clr_ax], 
                           fig_margin={'top':0, 'bottom':0, 'left':0, 'right':0})
line_fig = bqplot.Figure(marks=[lines], axes=[x_ax, y_ax])

us_map_fig.layout.width = '500px'
line_fig.layout.width = '500px'

# Dashboard
my_dashboard = ipywidgets.HBox([us_map_fig, line_fig])
my_dashboard

In [33]:
df_selected = df.loc[(df['country']=='us')&(df['state'].isin(['nm', 'tx']))]
df_selected_g = df_selected.groupby(df_selected['date_sighted'].dt.year)[['duration']].sum()
df_selected_g

,duration
date_sighted,
1969,180.0
1982,60.0
1994,120.0
1995,600.0
1997,900.0
2000,1208.0
2001,7322.0
2002,900.0
2003,1200.0


In [34]:
# Click select + sum of duration in seconds (the selected states presented as seperated lines)
# Left plot
# Scale
geo_sc = bqplot.AlbersUSA()
clr_sc = bqplot.ColorScale(scheme='Oranges')

# Axis
clr_ax = bqplot.ColorAxis(scale=clr_sc, side='top')

# Mark
us_map = bqplot.Map(map_data = us_map_data, 
                    scales={'projection':geo_sc, 'color':clr_sc}, 
                    color = clr, 
                    interactions ={'click': 'select'}, 
                    selected_styles={'selected_fill':'green'})

# Right: Line plot
# Scale
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Years')
y_ax = bqplot.Axis(scale=y_sc, label='Sum of duration in seconds', orientation='vertical')

# Marks
lines = bqplot.Lines(scales={'x':x_sc, 'y':y_sc}, 
                     display_legend=True)


# Interaction
def select_func(change):
    selected_fips = us_map.selected
    print(selected_fips)
    
    if selected_fips is not None:
    
        state_abbrs = states_df.loc[states_df['fips'].isin(selected_fips)]['name_abbr'].tolist()
        print(state_abbrs)

        line_x_vals = []
        line_y_vals = []
        legend_lbl = []
        
        for state_abbr in state_abbrs:
            df_selected = df.loc[(df['country']=='us')&(df['state']==state_abbr)]
            df_selected_g = df_selected.groupby(df_selected['date_sighted'].dt.year)[['duration']].sum()
            
            x_val_4_a_line = df_selected_g.index.tolist()
            y_val_4_a_line = df_selected_g['duration'].tolist()
            
            line_x_vals.append(x_val_4_a_line)
            line_y_vals.append(y_val_4_a_line)
            
            state_full = states_df.loc[states_df['name_abbr']==state_abbr]['name_full'].tolist()[0]
            legend_lbl.append(state_full)
            
        lines.x = line_x_vals
        lines.y = line_y_vals
        lines.labels = legend_lbl
    
    else:
        lines.x = []
        lines.y = []

us_map.observe(select_func, 'selected')

# Fig
us_map_fig = bqplot.Figure(marks=[us_map], axes=[clr_ax], 
                           fig_margin={'top':0, 'bottom':0, 'left':0, 'right':0})
line_fig = bqplot.Figure(marks=[lines], axes=[x_ax, y_ax], 
                         legend_location='top-left', 
                         legend_style={'fill':'white'})

us_map_fig.layout.width = '500px'
line_fig.layout.width = '500px'

# Dashboard
my_dashboard = ipywidgets.HBox([us_map_fig, line_fig])
my_dashboard


## A Dashboard with three plots

[Matplotlib colormap](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

```
OrdinalColorScale()

ordinal_schemes = {
    'Set2': 8,
    'Accent': 8,
    'Set1': 9,
    'Set3': 12,
    'Dark2': 8,
    'Paired': 12,
    'Pastel2': 8,
    'Pastel1': 9,
}
```

In [35]:
# Click select + sum of duration in seconds (the selected states are in seperated lines) + stacked bars by shapes

# Top plot : Map
# Scale
geo_sc = bqplot.AlbersUSA()
clr_sc = bqplot.ColorScale(scheme='Oranges')

# Axis
clr_ax = bqplot.ColorAxis(scale=clr_sc, side='top')

# Mark
us_map = bqplot.Map(map_data = us_map_data, 
                    scales={'projection':geo_sc, 'color':clr_sc}, 
                    color = clr, 
                    interactions ={'click': 'select'}, 
                    selected_styles={'selected_fill':'green'})

# Bottom, Left Line plot
# Scale
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

subplot_c_sc = bqplot.OrdinalColorScale(scheme='Dark2')

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Years')
y_ax = bqplot.Axis(scale=y_sc, label='Sum of duration in seconds', orientation='vertical')

# Marks
lines = bqplot.Lines(scales={'x':x_sc, 'y':y_sc, 'color':subplot_c_sc}, 
                     display_legend=True)

# Bottom, Left: Stacked bar
# Scale
x_sc_r = bqplot.OrdinalScale()
y_sc_r = bqplot.LinearScale()

# Axis
x_ax_r = bqplot.Axis(scale=x_sc_r, label='UFO shapes')
y_ax_r = bqplot.Axis(scale=y_sc_r, label='UFO counts', orientation='vertical')

# Mark
bars = bqplot.Bars(scales={'x':x_sc_r, 'y':y_sc_r, 'color': subplot_c_sc}, 
                   type='stacked', color_mode='element', 
                   display_legend=True, 
                   base=0)


# Interaction
def select_func(change):
    selected_fips = us_map.selected
    print(selected_fips)
    
    if selected_fips is not None:
    
        state_abbrs = states_df.loc[states_df['fips'].isin(selected_fips)]['name_abbr'].tolist()
        print(state_abbrs)
        
        subplot_clrs = np.arange(1, len(state_abbrs)+1)

        line_x_vals = []
        line_y_vals = []
        legend_lbl = []
        
        # Line plot
        for state_abbr in state_abbrs:
            df_selected = df.loc[(df['country']=='us')&(df['state']==state_abbr)]
            df_selected_g = df_selected.groupby(df_selected['date_sighted'].dt.year)[['duration']].sum()
            
            x_val_4_a_line = df_selected_g.index.tolist()
            y_val_4_a_line = df_selected_g['duration'].tolist()
            
            line_x_vals.append(x_val_4_a_line)
            line_y_vals.append(y_val_4_a_line)
            
            state_full = states_df.loc[states_df['name_abbr']==state_abbr]['name_full'].tolist()[0]
            legend_lbl.append(state_full)
            
        lines.x = line_x_vals
        lines.y = line_y_vals
        lines.labels = legend_lbl
        lines.color = subplot_clrs
        
        # Bar chart
        bar_y_vals = []
        
        df_selected_bar = df.loc[(df['country']=='us')&(df['state'].isin(state_abbrs))]
        df_selected_bar_g = df_selected_bar.groupby(['state', 'shape'])[['ufo_id']].count()
        df_selected_bar_g = df_selected_bar_g.reset_index()
        
        shapes = df_selected_bar_g['shape'].unique().tolist()
        
        for state_abbr in state_abbrs:
            state_y_vals = []
            for shape in shapes:
                filter_1 = df_selected_bar_g['state'] == state_abbr
                filter_2 = df_selected_bar_g['shape'] == shape
                
                selected_y_val = df_selected_bar_g.loc[(filter_1)&(filter_2)]
                
                y_val = 0
                if selected_y_val.shape[0] > 0:
                    y_val = selected_y_val['ufo_id'].tolist()[0]
                
                state_y_vals.append(y_val)
        
            bar_y_vals.append(state_y_vals)
        
        print(shapes)
        print(bar_y_vals)
        bars.x = shapes
        bars.y = bar_y_vals
        bars.labels = legend_lbl
        bars.color = subplot_clrs
    
    else:
        lines.x = []
        lines.y = []
        
        bars.x = []
        bars.y = []

us_map.observe(select_func, 'selected')

# Fig
us_map_fig = bqplot.Figure(marks=[us_map], axes=[clr_ax], 
                           fig_margin={'top':0, 'bottom':0, 'left':0, 'right':0})
line_fig = bqplot.Figure(marks=[lines], axes=[x_ax, y_ax], 
                         legend_location='top-left', 
                         legend_style={'fill':'white'})
bar_fig = bqplot.Figure(marks=[bars], axes=[x_ax_r, y_ax_r], 
                        legend_location='top-left', 
                        legend_style={'fill':'white'})

#us_map_fig.layout.width = '500px'
line_fig.layout.width = '500px'
bar_fig.layout.width = '500px'
# Dashboard
bottom_panel = ipywidgets.HBox([line_fig, bar_fig])
my_dashboard = ipywidgets.VBox([us_map_fig, bottom_panel])
my_dashboard

[48]
['tx']
['chevron', 'circle', 'cylinder', 'disk', 'egg', 'fireball', 'flash', 'formation', 'light', 'other', 'oval', 'rectangle', 'sphere', 'teardrop', 'triangle', 'unknown']
[[2, 4, 1, 2, 2, 1, 1, 1, 8, 5, 3, 1, 2, 1, 3, 4]]
[48 12]
['fl', 'tx']
['circle', 'cylinder', 'disk', 'fireball', 'formation', 'light', 'other', 'sphere', 'triangle', 'unknown', 'chevron', 'egg', 'flash', 'oval', 'rectangle', 'teardrop']
[[4, 1, 6, 7, 1, 9, 3, 4, 3, 3, 0, 0, 0, 0, 0, 0], [4, 1, 2, 1, 1, 8, 5, 2, 3, 4, 2, 2, 1, 3, 1, 1]]


In [36]:
df_selected_bar = df.loc[(df['country']=='us')&(df['state'].isin(['il', 'az']))]
df_selected_bar_g = df_selected_bar.groupby(['state', 'shape'])[['ufo_id']].count()
df_selected_bar_g = df_selected_bar_g.reset_index()
df_selected_bar_g

,state,shape,ufo_id
0,az,chevron,1
1,az,cigar,1
2,az,circle,7
3,az,disk,2
4,az,fireball,5
5,az,flash,1
6,az,light,3
7,az,other,4
8,az,oval,2
9,az,sphere,7


In [37]:
filter_1 = df_selected_bar_g['state'] == 'il'
filter_2 = df_selected_bar_g['shape'] == 'chevron'
df_selected_bar_g.loc[(filter_1)&(filter_2)]

,state,shape,ufo_id
